In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from tqdm import tqdm

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Load the CommonsenseQA dataset
dataset = load_dataset("commonsense_qa")

# Load DeBERTa tokenizer and model
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMultipleChoice.from_pretrained(model_name, num_labels=5)
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

...

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForMultipleChoice(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
 

In [ ]:
# Check structure of the dataset
print(dataset['train'][5])

{'id': 'e8a8b3a2061aa0e6d7c6b522e9612824', 'question': 'What home entertainment equipment requires cable?', 'question_concept': 'cable', 'choices': {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['radio shack', 'substation', 'cabinet', 'television', 'desk']}, 'answerKey': 'D'}


In [ ]:
def preprocess_function(examples):
    questions = examples['question']
    choices = examples['choices']
    num_choices = len(choices[0]['text'])
    # print(num_choices)

    # print([choice for choice_set in choices for choice in choice_set['text']])
    # Flatten the choices
    flat_choices = [choice for choice_set in choices for choice in choice_set['text']]

    # Repeat each question for each choice
    repeated_questions = [[q] * num_choices for q in questions]
    flat_questions = [q for sublist in repeated_questions for q in sublist]

    # Combine questions and choices
    inputs = [f"Question: {q} Choice: {c}" for q, c in zip(flat_questions, flat_choices)]

    # Tokenize
    tokenized_inputs = tokenizer(inputs, truncation=True, padding='max_length', max_length=128)

    # Reshape the inputs
    tokenized_inputs = {k: [v[i:i+num_choices] for i in range(0, len(v), num_choices)] for k, v in tokenized_inputs.items()}

    labels = []
    for ans in examples['answerKey']:
        if ans and 'A' <= ans <= 'E':  # Check if answer is valid and within range
            labels.append(ord(ans) - ord('A'))
        else:
            labels.append(0)  # Default to 0 for invalid answers

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 9741
    })
    validation: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1221
    })
    test: Dataset({
        features: ['id', 'question', 'question_concept', 'choices', 'answerKey', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1140
    })
})

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./deberta-finetuned",  # Directory to save the model
    evaluation_strategy="epoch",      # Evaluate after every epoch
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,               # Save only the last 2 checkpoints
    load_best_model_at_end=True,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['validation'],
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-bf435d80ab28>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.846000,0.626029
2,0.470200,0.696905
3,0.281900,0.943071


TrainOutput(global_step=3654, training_loss=0.5317859649658203, metrics={'train_runtime': 3936.5808, 'train_samples_per_second': 7.423, 'train_steps_per_second': 0.928, 'total_flos': 9611204033399040.0, 'train_loss': 0.5317859649658203, 'epoch': 3.0})

In [ ]:
# Evaluate on the validation set
results = trainer.evaluate(encoded_dataset['validation'])
print(f"Validation Results: {results}")

Validation Results: {'eval_loss': 0.626029372215271, 'eval_runtime': 46.9589, 'eval_samples_per_second': 26.001, 'eval_steps_per_second': 3.258, 'epoch': 3.0}


In [ ]:
def predict_answer(model, tokenizer, question, choices):
    # Prepare inputs similar to how they were prepared during training
    # Each choice forms a separate input sequence
    inputs = [f"Question: {question} Choice: {choice}" for choice in choices]

    # Tokenize all choice sequences
    encoding = tokenizer(
        inputs,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt"
    )

    # The model expects inputs in shape: (batch_size, num_choices, seq_length)
    # Here we have one "batch" with multiple choices, so we add a batch dimension
    for key in encoding:
        encoding[key] = encoding[key].unsqueeze(0)  # Now shape: (1, num_choices, seq_length)

    # Move model and inputs to appropriate device (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    encoding = {k: v.to(device) for k, v in encoding.items()}

    # Predict without gradient updates
    with torch.no_grad():
        outputs = model(**encoding)

    # The model outputs logits of shape (batch_size=1, num_choices)
    # Choose the answer with the highest logit score
    predicted_label = outputs.logits.argmax(dim=1).item()
    predicted_answer = choices[predicted_label]

    return predicted_answer



In [ ]:
# take 5 samples from training set and print correct answer and predicted answer
sample_questions = dataset['train'].select(range(5))
for sample in sample_questions:
    question = sample['question']
    choices = sample['choices']['text']
    correct_answer = sample['answerKey']
    correct_answer = choices[ord(correct_answer) - ord('A')]
    predicted_answer = predict_answer(model, tokenizer, question, choices)
    print(f"Question: {question}")
    print(f"Choices: {choices}")
    print(f"Correct Answer: {correct_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("-"*50)

Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
Choices: ['ignore', 'enforce', 'authoritarian', 'yell at', 'avoid']
Correct Answer: ignore
Predicted Answer: ignore
--------------------------------------------------
Question: Sammy wanted to go to where the people were.  Where might he go?
Choices: ['race track', 'populated areas', 'the desert', 'apartment', 'roadblock']
Correct Answer: populated areas
Predicted Answer: populated areas
--------------------------------------------------
Question: To locate a choker not located in a jewelry box or boutique where would you go?
Choices: ['jewelry store', 'neck', 'jewlery box', 'jewelry box', 'boutique']
Correct Answer: jewelry store
Predicted Answer: jewelry store
--------------------------------------------------
Question: Google Maps and other highway and street GPS services have replaced what?
Choices: ['united states', 'mexico', 'countryside', 'atlas', 

In [ ]:
# take 5 samples from validation set and print correct answer and predicted answer
sample_questions = dataset['validation'].select(range(5))
for sample in sample_questions:
    question = sample['question']
    choices = sample['choices']['text']
    correct_answer = sample['answerKey']
    correct_answer = choices[ord(correct_answer) - ord('A')]
    predicted_answer = predict_answer(model, tokenizer, question, choices)
    print(f"Question: {question}")
    print(f"Choices: {choices}")
    print(f"Correct Answer: {correct_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print("-"*50)

Question: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?
Choices: ['bank', 'library', 'department store', 'mall', 'new york']
Correct Answer: bank
Predicted Answer: mall
--------------------------------------------------
Question: What do people aim to do at work?
Choices: ['complete job', 'learn from each other', 'kill animals', 'wear hats', 'talk to each other']
Correct Answer: complete job
Predicted Answer: complete job
--------------------------------------------------
Question: Where would you find magazines along side many other printed works?
Choices: ['doctor', 'bookstore', 'market', 'train station', 'mortuary']
Correct Answer: bookstore
Predicted Answer: bookstore
--------------------------------------------------
Question: Where are  you likely to find a hamburger?
Choices: ['fast food restaurant', 'pizza', 'ground up dead cows', 'mouth', 'cow carcus']
Correct Answer: fast food restaurant
Predicted Answer: fast foo

In [ ]:
# take 5 samples from testing set and print predicted answer
sample_questions = dataset['test'].select(range(5))
for sample in sample_questions:
    question = sample['question']
    choices = sample['choices']['text']
    correct_answer = sample['answerKey']
    correct_answer = choices[ord(correct_answer) - ord('A')]
    predicted_answer = predict_answer(model, tokenizer, question, choices)
    print(f"Question: {question}")
    print(f"Choices: {choices}")
    print(f"Predicted Answer: {predicted_answer}")
    print("-"*50)

Question: The townhouse was a hard sell for the realtor, it was right next to a high rise what?
Choices: ['suburban development', 'apartment building', 'bus stop', 'michigan', 'suburbs']
Predicted Answer: apartment building
--------------------------------------------------
Question: There is a star at the center of what group of celestial bodies?
Choices: ['hollywood', 'skyline', 'outer space', 'constellation', 'solar system']
Predicted Answer: constellation
--------------------------------------------------
Question: What were the kids doing as they looked up at the sky and clouds?
Choices: ['ponder', 'become adults', 'wonder about', 'open door', 'distracting']
Predicted Answer: wonder about
--------------------------------------------------
Question: The person taught an advanced class only for who?
Choices: ['own house', 'own self', 'wonderful memories', 'know truth', 'intelligent children']
Predicted Answer: intelligent children
--------------------------------------------------
Q

In [ ]:
def calculate_validation_score(model, tokenizer, dataset):
    total_correct = 0
    total_questions = len(dataset['validation'])
    incorrect_instances = []

    for instance in tqdm(dataset['validation'], desc="Validating"):
        question = instance['question']
        choices = instance['choices']['text']
        correct_answer = instance['answerKey']
        correct_answer = choices[ord(correct_answer) - ord('A')]
        predicted_answer = predict_answer(model, tokenizer, question, choices)

        if predicted_answer == correct_answer or predicted_answer in correct_answer:
            total_correct += 1
        else:
            incorrect_instances.append(instance)

    validation_score = total_correct / total_questions

    return validation_score, incorrect_instances

val_score, incorrect_ins = calculate_validation_score(model, tokenizer, dataset)


Validating: 100%|██████████| 1221/1221 [01:23<00:00, 14.68it/s]


In [ ]:
print(f"Total Correct Answers: {val_score*len(dataset['validation'])}")
print(f"Total Questions: {len(dataset['validation'])}")
print(f"Incorrect Answers: {round((1-val_score)*len(dataset['validation']))}")
print(f"Validation Score: {val_score*100:.2f}")

Total Correct Answers: 949.0
Total Questions: 1221
Incorrect Answers: 272
Validation Score: 77.72


In [ ]:
incorrect_ins[:5]

[{'id': '1afa02df02c908a558b4036e80242fac',
  'question': 'A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?',
  'question_concept': 'revolving door',
  'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
   'text': ['bank', 'library', 'department store', 'mall', 'new york']},
  'answerKey': 'A'},
 {'id': '1d24f406b6828492040b405d3f35119c',
  'question': 'What do people typically do while playing guitar?',
  'question_concept': 'playing guitar',
  'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
   'text': ['cry', 'hear sounds', 'singing', 'arthritis', 'making music']},
  'answerKey': 'C'},
 {'id': 'c5c4166f2ed3c2b3517b79e6848e9ae2',
  'question': 'When drinking booze what can you do to stay busy?',
  'question_concept': 'booze',
  'choices': {'label': ['A', 'B', 'C', 'D', 'E'],
   'text': ['reach tentative agreement',
    'stay in bed',
    'stop bicycle',
    'examine thing',
    'suicide']},
  'answerKey': 'D'},
 {'id': '6dc5b288

In [ ]:
model.save_pretrained("./final_model/deberta-finetuned")
tokenizer.save_pretrained("./final_model/deberta-finetuned")

('./final_model/deberta-finetuned/tokenizer_config.json',
 './final_model/deberta-finetuned/special_tokens_map.json',
 './final_model/deberta-finetuned/spm.model',
 './final_model/deberta-finetuned/added_tokens.json',
 './final_model/deberta-finetuned/tokenizer.json')

In [ ]:
# loaded_model = AutoModelForMultipleChoice.from_pretrained("./deberta-finetuned")
# loaded_tokenizer = AutoTokenizer.from_pretrained("./deberta-finetuned")